In [1]:
# 10 models each
# logit
# 0
# 0.1
# 0.5
# 1
# 5
# 10

import pandas as pd
import numpy as np

from fit_adversarial import fit_adversarial_regressor, test_model
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("all_vals_race1_interestrate1.csv")
from sklearn.model_selection import train_test_split

df.drop(
    columns=[
        "IsTestData",
        "IsCalibrateData",
        "v1",
        "istestdata2",
        "iscalibratedata2",
    ],
    inplace=True,
)


df["minority"] = (
    (df["race_dum_Black"] == 1)
    | (df["race_dum_Native Am, Alaska, Hawaii"] == 1)
    | (df["race_dum_White hisp"] == 1)
)

race_columns = [
    "race_dum_Unknown",
    "race_dum_Asian",
    "race_dum_Black",
    "race_dum_Native Am, Alaska, Hawaii",
    "race_dum_White hisp",
    "race_dum_White_non_hisp",
]

ethnicity_column = "ethnicity_dum_hisp"

features = df.drop(
    columns=["default", "cur_int_rate", "sato", "default_age"] + race_columns + [ethnicity_column],
    axis=1,
)

target = df["default"]


sensitive_features = df[["minority"]].values

C:\Users\bencu\AppData\Local\Temp\ipykernel_19412\107014848.py:16: DtypeWarning: Columns (101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_vals_race1_interestrate1.csv")


In [2]:
import math

features = features.applymap(
    lambda x: x if isinstance(x, (int, float)) and not math.isnan(x) else 0
)

C:\Users\bencu\AppData\Local\Temp\ipykernel_19412\2393365648.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features = features.applymap(


In [3]:
X_train, X_val, y_train, y_val, sensitive_train, sensitive_val = train_test_split(
    features, target, sensitive_features, test_size=0.3, random_state=42
)

In [ ]:
import json
from datetime import datetime
from pathlib import Path
import numpy as np


alphas = ["logit", 0, 0.1, 0.5, 1, 5, 10]
runs_per_alpha = 10
output_path = Path("metrics_results_default.jsonl")

def _to_py(value):
    if isinstance(value, (np.ndarray,)):
        return value.tolist()
    if isinstance(value, (np.floating, np.integer)):
        return value.item()
    return value

def serialize_results(alpha_label, run_idx, dataset_label, results_dict):
    out = {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "alpha": alpha_label,
        "run": run_idx,
        "dataset": dataset_label,
    }
    for k in [
        "accuracy",
        "precision",
        "roc_auc",
        "confusion_matrix",
        "classification_report",
    ]:
        if k in results_dict and results_dict[k] is not None:
            out[k] = _to_py(results_dict[k])
    fm = results_dict.get("fairness_metrics")
    if isinstance(fm, dict):
        mean_metrics = fm.get("metrics", {}).get("mean_metrics")
        std_metrics = fm.get("metrics", {}).get("std_metrics")
        roc_curves = fm.get("roc_curves")
        out["fairness_mean_metrics"] = _to_py(mean_metrics) if mean_metrics is not None else None
        out["fairness_std_metrics"] = _to_py(std_metrics) if std_metrics is not None else None
        out["fairness_roc_curves"] = _to_py(roc_curves) if roc_curves is not None else None
    return out

output_path.touch(exist_ok=True)

baseline_m, baseline_s = fit_adversarial_regressor(
    alpha=0,
    X_train=X_train,
    X_val=X_val,
    y_train=y_train,
    y_val=y_val,
    sensitive_train=sensitive_train,
    sensitive_val=sensitive_val,
    filename=None,
    epochs=12,
    use_skip_connections=False,
    is_regression=False,
    baseline="logit",
)

baseline_all = test_model(
    baseline_m,
    baseline_s,
    features,
    target,
    sensitive_features,
    verbose=False,
    is_regression=False,
)

baseline_val = test_model(
    baseline_m,
    baseline_s,
    X_val,
    y_val,
    sensitive_val,
    verbose=False,
    is_regression=False
)

with output_path.open("a", encoding="utf-8") as f:
    f.write(json.dumps(serialize_results("logit", 0, "all", baseline_all)) + "\n")
    f.write(json.dumps(serialize_results("logit", 0, "val", baseline_val)) + "\n")

del baseline_m
del baseline_s

for alpha in [0, 0.1, 0.5, 1, 5, 10]:
    for run in range(1, runs_per_alpha + 1):
        mitigator, scaler = fit_adversarial_regressor(
            alpha=alpha,
            X_train=X_train,
            X_val=X_val,
            y_train=y_train,
            y_val=y_val,
            sensitive_train=sensitive_train,
            sensitive_val=sensitive_val,
            filename=None,
            epochs=12,
            use_skip_connections=False,
            is_regression=False,
)
        res_all = test_model(
            mitigator,
            scaler,
            features,
            target,
            sensitive_features,
            verbose=False,
            is_regression=False,
)
        res_val = test_model(
            mitigator,
            scaler,
            X_val,
            y_val,
            sensitive_val,
            verbose=False,
            is_regression=False
)
        with output_path.open("a", encoding="utf-8") as f:
            f.write(json.dumps(serialize_results(alpha, run, "all", res_all)) + "\n")
            f.write(json.dumps(serialize_results(alpha, run, "val", res_val)) + "\n")
        del mitigator
        del scaler

Epoch 1/12, Average adversary loss: 0.499425. Average predictor loss: 0.199640
Current alpha 0, learning_rate 0.001
Validation - Epoch 1/12, Adversary Loss: 5.205425, Predictor Loss: 1.597829
Epoch 2/12, Average adversary loss: 0.497792. Average predictor loss: 0.194021
Current alpha 0, learning_rate 0.001
Validation - Epoch 2/12, Adversary Loss: 5.118193, Predictor Loss: 1.543139
Epoch 3/12, Average adversary loss: 0.492594. Average predictor loss: 0.190261
Current alpha 0, learning_rate 0.001
Validation - Epoch 3/12, Adversary Loss: 5.032048, Predictor Loss: 1.560441
Epoch 4/12, Average adversary loss: 0.493277. Average predictor loss: 0.189129
Current alpha 0, learning_rate 0.001
Validation - Epoch 4/12, Adversary Loss: 4.966121, Predictor Loss: 1.561764
Epoch 5/12, Average adversary loss: 0.493800. Average predictor loss: 0.189211
Current alpha 0, learning_rate 0.001
Validation - Epoch 5/12, Adversary Loss: 5.012915, Predictor Loss: 1.540006
Epoch 6/12, Average adversary loss: 0.49